## Investigate MetaKB Dataset: Therapeutics & Combination Therapies
In order to better understand the MetaKB aggregate dataset, perform graph-directed lookups via neo4j to identify potential insights or avenues of discussion that might not previously be accessible.
  
This notebook focuses on exploring the Therapeutics & Combination Therapies datasets across different variants, studies, and gene contexts. Initial ideas are to look at FDA approval for therapies, diseases in use, and off-label use cases on a variant to variant or gene to gene basis.
  
**Current Data Version**: 5.20.0

In [215]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

### Grab Therapeutics Data

In [197]:
from neo4j import GraphDatabase

# Function to create a connection to the Neo4j database
def create_db_connection(uri, user, password):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    return driver

# Function to execute a Cypher query
def execute_query(driver, query):
    with driver.session() as session:
        result = session.run(query)
        return [record for record in result]

# Connect to the Neo4j database
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"  # Replace 'your_password' with your actual password
driver = create_db_connection(uri, user, password)

# Strict, Must have Combination Therapies
query = """MATCH (s:Study)-[:HAS_VARIANT]->(variant),
      (s)-[:HAS_THERAPEUTIC]->(therapeutic),
      (therapeutic)-[:HAS_COMPONENTS]->(combination)
RETURN properties(s) AS Study, 
       properties(variant) AS VariantProperties,
       properties(therapeutic) AS TherapeuticProperties,
       COUNT(combination) AS NumberOfComponents,
       COLLECT(properties(combination)) AS CombinationTherapyProperties
"""

query_therapies = """
MATCH (s:Study)-[:HAS_VARIANT]->(variant),
      (s)-[:HAS_THERAPEUTIC]->(therapeutic)
OPTIONAL MATCH (therapeutic)-[:HAS_COMPONENTS]->(combination)
RETURN properties(s) AS Study, 
       properties(variant) AS VariantProperties,
       properties(therapeutic) AS TherapeuticProperties,
       COUNT(combination) AS NumberOfComponents,
       COLLECT(properties(combination)) AS CombinationTherapyProperties
"""

# Execute the query
result = execute_query(driver, query_therapies)

# Close the connection
driver.close()




In [199]:
import pandas as pd

data = []
for record in result:
    row = {
        'study_allele_origin': record['Study'].get('alleleOrigin', None),
        'study_id': record['Study']['id'],
        'study_direction': record['Study']['direction'],
        'study_predicate': record['Study']['predicate'],
        'study_type': record['Study']['type'],
        'variant_mp_score': record['VariantProperties'].get('civic_molecular_profile_score', None),
        'variant_id': record['VariantProperties']['id'],
        'variant_label': record['VariantProperties']['label'],
        'variant_type': record['VariantProperties'].get('variant_types',None),
        'therapeutic_type': record['TherapeuticProperties']['type'],
        'therapeutic_civic_type': record['TherapeuticProperties'].get('civic_therapy_interaction_type', None),
        'therapeutic_id': record['TherapeuticProperties']['id'],
        'therapeutic_label': record['TherapeuticProperties'].get('label',None),
        'number_of_components': record.get('NumberOfComponents', None),
        'combination_therapy_components': record.get('CombinationTherapyProperties', None)
    }

    data.append(row)

df = pd.DataFrame(data)

df[0:5]

,study_allele_origin,study_id,study_direction,study_predicate,study_type,variant_mp_score,variant_id,variant_label,variant_type,therapeutic_type,therapeutic_civic_type,therapeutic_id,therapeutic_label,number_of_components,combination_therapy_components
0,somatic,civic.eid:238,supports,predictsResistanceTo,VariantTherapeuticResponseStudy,406.25,civic.mpid:34,EGFR T790M,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:15,Erlotinib,0,[]
1,somatic,civic.eid:1409,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,1378.50,civic.mpid:12,BRAF V600E,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:4,Vemurafenib,0,[]
2,somatic,civic.eid:1592,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,406.25,civic.mpid:34,EGFR T790M,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:187,Osimertinib,0,[]
3,somatic,civic.eid:1867,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,406.25,civic.mpid:34,EGFR T790M,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:187,Osimertinib,0,[]
4,somatic,civic.eid:2994,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,379.00,civic.mpid:33,EGFR L858R,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:15,Erlotinib,0,[]


In [200]:
# save
# df.to_excel('therapeutics.xlsx',sheet_name='Sheet1')

### Inspect

In [3]:
df['variant_mp_score'].describe()

count     815.000000
mean      176.373620
std       382.819215
min         0.500000
25%         7.500000
50%        20.000000
75%        96.500000
max      1378.500000
Name: variant_mp_score, dtype: float64

In [4]:
df['variant_label'].value_counts()

variant_label
BRAF V600E                 70
EGFR L858R                 36
BRAF p.V600E (Missense)    31
EGFR T790M                 27
PIK3CA H1047R              22
                           ..
KDR R961W                   1
ERBB2 G778_S779insLPS       1
ERBB2 G776delinsLC          1
ERBB2 G776delinsCV          1
IDH1 p.R132L (Missense)     1
Name: count, Length: 474, dtype: int64

In [5]:
df['number_of_components'].value_counts()

number_of_components
0    922
2    101
3     19
Name: count, dtype: int64

In [233]:
braf_df['variant_mp_score'].describe()

count     123.000000
mean      798.526423
std       669.656836
min         0.750000
25%        24.500000
50%      1378.500000
75%      1378.500000
max      1378.500000
Name: variant_mp_score, dtype: float64

### Distribution of Variant MP Scores
Upon manual inspection, I noticed that BRAF containing variants tended to have huge Variant MP Scores and wondered how this compared to the rest of the data set of gene variants with therapeutic data.

In [206]:
braf_df = df[df['variant_label'].str.contains('BRAF')]
not_braf_df = df[~df['variant_label'].str.contains('BRAF')]

combined_df = pd.concat([
    pd.DataFrame({'variant_mp_score': df['variant_mp_score'], 'source': 'Combined'}),
    pd.DataFrame({'variant_mp_score': braf_df['variant_mp_score'], 'source': 'BRAF'}),
    pd.DataFrame({'variant_mp_score': not_braf_df['variant_mp_score'], 'source': 'All Other'})
])

# Create a horizontal box plot with both datasets
fig = px.box(combined_df, x='variant_mp_score', color='source', title='Box and Whisker Plot of BRAF and Other Variant MP Scores')

# Update layout for better visuals
fig.update_layout(
    xaxis_title="Variant MP Score",
    boxmode='group'  # Shows the boxes side by side
)

# Show the plot
fig.show()
pio.write_image(fig, "Var_mp_score_dist_BRAF_vs_Other.png", format='png', width=1200, height=400, scale=5)

### Fetch Combination Therapy Components
Initial query grabbed just the Combo therapy ID. This additional query takes the ctid from each row and just quickly grabs the associated properties to fill out the dataset.

In [207]:

def neo4j_query(combo_id):
    # query for the specific therapy ID components
    query_components = f"""
    MATCH (c:CombinationTherapy)-[:HAS_COMPONENTS]-(t)
    WHERE c.id = \'{combo_id}\'
    RETURN properties(t) AS Therapeutic
    """

    # Execute the query
    result = execute_query(driver, query_components)

    # Close the connection
    driver.close()
    return(result)



In [208]:
def get_combination_therapies(tdf):
    tdf['combination_components'] = None

    for idx, row in tdf.iterrows():
        result = neo4j_query(row['therapeutic_id'])
        therapeutics = []
        for record in result:
            therapeutics.append(record['Therapeutic'].get('label', None))
            tdf.at[idx, 'combination_components'] = therapeutics

    for idx, row in tdf.iterrows():
        if row['therapeutic_type'] == 'CombinationTherapy':
            components = row['combination_components']
            if components is not None:  # Check if components is not None
                sorted_components = tuple(sorted(components))
                tdf.at[idx, 'combination_components'] = sorted_components
    return(tdf)

braf_df = get_combination_therapies(braf_df)
df = get_combination_therapies(df)
not_braf_df = get_combination_therapies(not_braf_df)

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_11898/1403308090.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_11898/1451180034.py:10: DeprecationWarning:

Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_11898/1451180034.py:10: DeprecationWarning:

Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_11898/1403308090.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [209]:
not_braf_df['combination_components'].value_counts()

combination_components
(Cetuximab, Irinotecan)                              12
(Dactolisib, Selumetinib)                             8
(Lapatinib, Neratinib, Trastuzumab)                   4
(Lapatinib, Trastuzumab)                              2
(Adagrasib, Cetuximab)                                2
(Cetuximab, Fluorouracil, Oxaliplatin)                2
(Cetuximab, Irinotecan, Vemurafenib)                  2
(Alpelisib, Fulvestrant)                              2
(Azacitidine, Ivosidenib)                             2
(Docetaxel, Selumetinib)                              2
(Capivasertib, Trastuzumab)                           1
(Durvalumab, Osimertinib)                             1
(Daunorubicin, Sunitinib)                             1
(Cytarabine, Sunitinib)                               1
(Cetuximab, Dactolisib)                               1
(Akt Inhibitor MK2206, Vemurafenib)                   1
(Lometrexol, Mercaptopurine)                          1
(Cisplatin, Pictilisib)  

### Quantify & Graph Therapeutic Application across BRAF/Non-BRAF Variants
While looking at all variant data in MetaKB that has an attached Therapeutic Application, quantify and graph the distribution of what therapeutic evidence is being applied where for BRAF vs non-BRAF variants

In [211]:
braf_df['therapeutic_label'].value_counts()

therapeutic_label
Vemurafenib                52
Trametinib                  9
Dabrafenib                  5
Encorafenib                 2
U0126                       2
Sorafenib                   2
Panitumumab                 2
Irinotecan                  2
Oxaliplatin                 2
Cetuximab                   2
MEK Inhibitor REC-4881      1
MEK Inhibitor RO4987655     1
BRAF Inhibitor              1
Pictilisib                  1
Cobimetinib                 1
Bleomycin                   1
Tovorafenib                 1
Name: count, dtype: int64

In [212]:
not_braf_df['therapeutic_label'].value_counts()

therapeutic_label
Erlotinib                 59
Imatinib                  43
Cetuximab                 37
Gefitinib                 31
Pictilisib                30
                          ..
JAK2 Inhibitor AZD1480     1
Patidegib                  1
PD173074                   1
AZD3463                    1
Larotrectinib              1
Name: count, Length: 113, dtype: int64

In [213]:
# Perform value_counts on the 'therapeutic_label' column
braf_counts = braf_df['therapeutic_label'].value_counts().rename_axis('therapeutic_label').reset_index(name='braf_count')
df_counts = df['therapeutic_label'].value_counts().rename_axis('therapeutic_label').reset_index(name='general_count')
not_braf_counts = not_braf_df['therapeutic_label'].value_counts().rename_axis('therapeutic_label').reset_index(name='not_braf_count')

# Combine the three DataFrames into one
combined_df = pd.merge(pd.merge(braf_counts, df_counts, on='therapeutic_label', how='outer'), not_braf_counts, on='therapeutic_label', how='outer').fillna(0)

# Convert counts to integers (optional, in case NaNs were filled)
combined_df[['braf_count', 'general_count', 'not_braf_count']] = combined_df[['braf_count', 'general_count', 'not_braf_count']].astype(int)

# Display the combined DataFrame
print(combined_df)


    therapeutic_label  braf_count  general_count  not_braf_count
0            AGI-5198           0              2               2
1             AZD3463           0              1               1
2         Abemaciclib           0              1               1
3           Adagrasib           0              2               2
4            Afatinib           0             19              19
..                ...         ...            ...             ...
118             U0126           2              2               0
119        Uprosertib           0              1               1
120       Vemurafenib          52             71              19
121        Venetoclax           0              4               4
122        Vismodegib           0              4               4

[123 rows x 4 columns]


In [218]:
# Add a new column for the total counts
combined_df['total_count'] = combined_df[['braf_count', 'general_count', 'not_braf_count']].sum(axis=1)

# Sort the DataFrame by the total counts in ascending order
sorted_df = combined_df.sort_values('total_count', ascending=False)

# Create a bar plot for each therapeutic label count
fig = px.bar(sorted_df, x='therapeutic_label', y=['braf_count', 'not_braf_count'], 
             title='Application of Therapeutic Evidence to Variants in MetaKB v2 (BRAF Variants vs All Others)',
             labels={'value': 'Count', 'variable': 'Dataset'},
             barmode='stack')

# Show the figure
fig.show()
pio.write_image(fig, "Application_of_Therapeutics_BRAF_vs_NonBRAF.png", format='png', width=2000, height=500, scale=5)


In [221]:
# Perform value_counts on the 'therapeutic_label' column
braf_counts = braf_df['combination_components'].value_counts().rename_axis('combination_components').reset_index(name='braf_count')
df_counts = df['combination_components'].value_counts().rename_axis('combination_components').reset_index(name='general_count')
not_braf_counts = not_braf_df['combination_components'].value_counts().rename_axis('combination_components').reset_index(name='not_braf_count')

# Combine the three DataFrames into one
combined_df = pd.merge(pd.merge(braf_counts, df_counts, on='combination_components', how='outer'), not_braf_counts, on='combination_components', how='outer').fillna(0)

# Convert counts to integers (optional, in case NaNs were filled)
combined_df[['braf_count', 'general_count', 'not_braf_count']] = combined_df[['braf_count', 'general_count', 'not_braf_count']].astype(int)

# Display the combined DataFrame
print(combined_df)


                               combination_components  braf_count  \
0                              (Adagrasib, Cetuximab)           0   
1                 (Akt Inhibitor MK2206, Vemurafenib)           0   
2                            (Alpelisib, Fulvestrant)           0   
3                           (Azacitidine, Ivosidenib)           0   
4   (BRAF Inhibitor, Mitogen-Activated Protein Kin...           2   
5            (Bevacizumab, Capecitabine, Oxaliplatin)           1   
6            (Bevacizumab, Capecitabine, Vemurafenib)           1   
7                           (Bevacizumab, Dabrafenib)           1   
8               (Binimetinib, Cetuximab, Encorafenib)           2   
9                          (Binimetinib, Encorafenib)           1   
10                          (Binimetinib, Everolimus)           0   
11                          (Capivasertib, Lapatinib)           0   
12        (Capivasertib, PI3K/BET Inhibitor LY294002)           0   
13                        (Capivas

In [225]:
# Add a new column for the total counts
combined_df['total_count'] = combined_df[['braf_count', 'general_count', 'not_braf_count']].sum(axis=1)

# Sort the DataFrame by the total counts in ascending order
sorted_df = combined_df.sort_values('total_count', ascending=False)
sorted_df['Labels'] = sorted_df['combination_components'].apply(lambda x: str(x))

# Create a bar plot for each therapeutic label count
fig = px.bar(sorted_df, x='Labels', y=['braf_count', 'not_braf_count'], 
             title='Application of Combination Therapeutics to Variant Evidence in MetaKB v2 (BRAF vs All Other)',
             labels={'value': 'Count', 'variable': 'Dataset'},
             barmode='stack')

# Show the figure
fig.show()
pio.write_image(fig, "Application_of_Combo_Therap_BRAF_vs_NonBRAF.png", format='png', width=2000, height=500, scale=5)


In [227]:

data = {
    'count': [sorted_df['braf_count'].sum(), sorted_df['not_braf_count'].sum()],
    'category': ['BRAF','Not BRAF'],
}
data

{'count': [62, 58], 'category': ['BRAF', 'Not BRAF']}

In [230]:
fig = px.pie(pd.DataFrame(data), values='count', names='category', title='Distribution of Combination Therapeutic Evidence for Variants in MetaKB v2')

fig.update_traces(textinfo='label+percent+value', 
                  hoverinfo='label+percent+value',
                  textposition='inside')

fig.show()
pio.write_image(fig, "Dist_Combo_Therapy_BRAF_v_NonBRAF.png", format='png', width=700, height=500, scale=5)


### Breakdown of Combination Therapeutic Evidence Application for Non-BRAF Variants, BRAF Variants
The fact that 51.7% of combination therapy evidence present in MetaKB v2 thus far is applied to BRAF based variant evidence was interesting. I wanted to follow-up and see of the non-BRAF variants, what genes are commonly applied combination therapy. Additionally, of the BRAF variant evidence, what specific variants are applied combination therapies.

In [231]:
# TODO: START HERE WITH TIDYING UP
not_braf_df[not_braf_df['therapeutic_type'].str.contains('CombinationTherapy')][0:3]

,study_allele_origin,study_id,study_direction,study_predicate,study_type,variant_mp_score,variant_id,variant_label,variant_type,therapeutic_type,therapeutic_civic_type,therapeutic_id,therapeutic_label,number_of_components,combination_therapy_components,combination_components
6,somatic,civic.eid:7318,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,50.0,civic.mpid:2579,PIK3CA H1047X,None,CombinationTherapy,COMBINATION,civic.ctid:QoPu2SO9N_zsHY4X6TMs5q3CLWJMHAQa,None,2,"[{'id': 'civic.tid:570', 'regulatory_approval'...","(Alpelisib, Fulvestrant)"
10,somatic,civic.eid:12063,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,176.5,civic.mpid:78,KRAS G12C,"[{""label"": ""missense_variant"", ""system"": ""http...",CombinationTherapy,COMBINATION,civic.ctid:qyRsTvZSTw7rBeyFy7j8bD0KfxiD5in9,None,2,"[{'id': 'civic.tid:16', 'regulatory_approval':...","(Adagrasib, Cetuximab)"
11,somatic,civic.eid:7315,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,75.5,civic.mpid:103,PIK3CA E542K,"[{""label"": ""missense_variant"", ""system"": ""http...",CombinationTherapy,COMBINATION,civic.ctid:QoPu2SO9N_zsHY4X6TMs5q3CLWJMHAQa,None,2,"[{'id': 'civic.tid:570', 'regulatory_approval'...","(Alpelisib, Fulvestrant)"


In [232]:
def grab_gene_symbol(entry):
    return(entry.split(' ')[0])

def grab_braf_var(entry):
    return(entry.split(' ')[1].replace('p.',''))

not_braf_df['gene_symbol'] = not_braf_df['variant_label'].apply(grab_gene_symbol)
braf_df['braf_variant'] = braf_df['variant_label'].apply(grab_braf_var)

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_11898/2565120429.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/5t/sfw5tjx56m10xb861_pd3wfm0000gq/T/ipykernel_11898/2565120429.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [180]:
not_braf_df[0:3]

,study_allele_origin,study_id,study_direction,study_predicate,study_type,variant_mp_score,variant_id,variant_label,variant_type,therapeutic_type,therapeutic_civic_type,therapeutic_id,therapeutic_label,number_of_components,combination_therapy_components,combination_components,gene_symbol
0,somatic,civic.eid:238,supports,predictsResistanceTo,VariantTherapeuticResponseStudy,406.25,civic.mpid:34,EGFR T790M,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:15,Erlotinib,0,[],None,EGFR
2,somatic,civic.eid:1592,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,406.25,civic.mpid:34,EGFR T790M,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:187,Osimertinib,0,[],None,EGFR
3,somatic,civic.eid:1867,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,406.25,civic.mpid:34,EGFR T790M,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:187,Osimertinib,0,[],None,EGFR


In [182]:
braf_df[0:3]

,study_allele_origin,study_id,study_direction,study_predicate,study_type,variant_mp_score,variant_id,variant_label,variant_type,therapeutic_type,therapeutic_civic_type,therapeutic_id,therapeutic_label,number_of_components,combination_therapy_components,combination_components,braf_variant
1,somatic,civic.eid:1409,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,1378.5,civic.mpid:12,BRAF V600E,"[{""label"": ""missense_variant"", ""system"": ""http...",TherapeuticAgent,None,civic.tid:4,Vemurafenib,0,[],None,V600E
8,somatic,civic.eid:9851,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,1378.5,civic.mpid:12,BRAF V600E,"[{""label"": ""missense_variant"", ""system"": ""http...",CombinationTherapy,COMBINATION,civic.ctid:P1PY89shAjemg7jquQ0V9pg1VnYnkPeK,None,2,"[{'id': 'civic.tid:16', 'regulatory_approval':...","(Cetuximab, Encorafenib)",V600E
9,somatic,civic.eid:3017,supports,predictsSensitivityTo,VariantTherapeuticResponseStudy,1378.5,civic.mpid:12,BRAF V600E,"[{""label"": ""missense_variant"", ""system"": ""http...",CombinationTherapy,COMBINATION,civic.ctid:oBrlcO23adoVXv51xh-5Wigy0QyDWtfr,None,2,"[{'id': 'civic.tid:19', 'regulatory_approval':...","(Dabrafenib, Trametinib)",V600E


In [185]:
tdf = not_braf_df[not_braf_df['therapeutic_type'].str.contains('CombinationTherapy')]

gene_counts = tdf['gene_symbol'].value_counts().rename_axis('gene_symbol').reset_index(name='count')
gene_counts

,gene_symbol,count
0,PIK3CA,18
1,KRAS,13
2,NRAS,6
3,ERBB2,5
4,GNAS,2
5,EGFR,2
6,FLT3,2
7,IDH1,2
8,ERCC2,1
9,ESR1,1


In [187]:
fig = px.bar(gene_counts, x='gene_symbol', y='count', 
             title='Genes with Applied Combination Therapy Strategies (non-BRAF)',
             labels={'gene_symbol': 'Gene', 'count': 'count'},
             barmode='stack',
             text_auto=True)

# Show the figure
fig.show()

In [195]:
tdf = braf_df[braf_df['therapeutic_type'].str.contains('CombinationTherapy')]


braf_variant_counts = tdf['braf_variant'].value_counts().rename_axis('variant').reset_index(name='count')
braf_variant_counts

,variant,count
0,V600E,51
1,V600K,5
2,D594G,2
3,V600D,1
4,A598V,1
5,G466V,1
6,G596C,1


In [196]:
fig = px.bar(braf_variant_counts, x='variant', y='count', 
             title='BRAF variants with Applied Combination Therapy Strategies',
             labels={'variant': 'Variant', 'count': 'count'},
             barmode='stack',
             text_auto=True)

# Show the figure
fig.show()